In [1]:
import pandas as pd
import numpy as np

In [12]:
# multi index from tuples:

index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop


(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [13]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [14]:
# Let's reindex with the new MultiIndex:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [6]:
# accessing multiindexed values:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [7]:
# unstacking multiindexed array into a DataFrame
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [8]:
# Naturally, the stack() method provides the opposite operation:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [9]:
# adding another layer (index, dimension)
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [10]:
pop_df.stack()

California  2000  total      33871648
                  under18     9267089
            2010  total      37253956
                  under18     9284094
New York    2000  total      18976457
                  under18     4687374
            2010  total      19378102
                  under18     4318033
Texas       2000  total      20851820
                  under18     5906301
            2010  total      25145561
                  under18     6879014
dtype: int64

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [12]:
########       Methods of MultiIndex Creation      ########3

# 1. pass two or more index arrays:

df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.290098  0.016110
  2  0.734068  0.261920
b 1  0.747384  0.370547
  2  0.120001  0.171249

In [2]:
# 2. pass a dictionary with appropriate tuples as keys
# Pandas will automatically recognize this and use a MultiIndex by default

data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [5]:
#------------- Explicit MultiIndex constructors ------------#

pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [6]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [8]:
# from a Cartesian product of single indices:

pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [10]:
### ---------- MultiIndex level names -----------------

In [15]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [16]:
# MultiIndex for columns    in Pandas:

# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      11.0  35.9  37.0  36.2  45.0  38.8
     2      47.0  35.0  38.0  36.8  35.0  37.3
2014 1      26.0  36.8  28.0  36.9  35.0  37.4
     2      35.0  37.4  37.0  38.0  22.0  37.9

In [47]:
# Indeed this is a four dimentional data: (subject, measurement type, year, visit)
# We can extract, e.g., a DataFrame for each subject (person):

health_data['Guido']

type          HR  Temp
year visit            
2013 1      37.0  36.2
     2      38.0  36.8
2014 1      28.0  36.9
     2      37.0  38.0

In [51]:
######3   Indexing and Slicing a MultiIndex ############3

pop['California', 2000]
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [54]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [55]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [56]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [57]:
##################### Multiply indexed DataFrames

In [58]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      11.0  35.9  37.0  36.2  45.0  38.8
     2      47.0  35.0  38.0  36.8  35.0  37.3
2014 1      26.0  36.8  28.0  36.9  35.0  37.4
     2      35.0  37.4  37.0  38.0  22.0  37.9

In [59]:
health_data['Guido', 'HR']

year  visit
2013  1        37.0
      2        38.0
2014  1        28.0
      2        37.0
Name: (Guido, HR), dtype: float64

In [60]:
# loc, iloc, and ix indexers 

health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      11.0  35.9
     2      47.0  35.0